#### jupyter启动步骤
1. 在pycharm中点击jupyter的|第一个代码块
2. 在右下方“Event Log”中点击“Open in Browser”

In [1]:
from pyecharts import Bar
import matplotlib
from datetime import datetime
from talib import MOM
import os
import pandas as pd
import alphalens as al
from pylab import mpl
import matplotlib.pyplot as plt

# %matplotlib inline

# 正常显示画图时出现的中文和负号
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['axes.unicode_minus'] = False

print(al.__version__)

0.3.6+8.g998e36d.dirty


### 数据格式要求

#### 因子数据
1. 需要处理成 Dataframe 格式，且其索引必须是二维日期和股票代码

#### 价格数据
1. 列为时间，行为股票的 Dataframe

#### 行业数据
1. 可有可无
2. 若有则可以设置成Dict格式或者Series格式

In [2]:
father_dir = os.path.abspath(os.path.dirname(os.getcwd()))  # 上一级目录
grandfather_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))  # 上上级目录
prices_path = os.path.join(father_dir, 'data', 'processed', 'prices\\')
factors_path = os.path.join(father_dir, 'data', 'processed', 'factors\\')

daily_prices = pd.read_csv(os.path.join(prices_path, 'daily_prices.csv'), parse_dates=True, index_col=['date'])
daily_factors = pd.read_csv(os.path.join(factors_path, 'daily_factors_new.csv'), parse_dates=True, index_col=['date', 'asset'])

parser = lambda dates: datetime.strptime(dates, '%Y-%m-%d %H:%M:%S')
# min_prices = pd.read_csv(os.path.join(prices_path, 'min_prices.csv'), parse_dates=True, index_col=['date'])
# min_factors = pd.read_csv(os.path.join(factors_path, 'min_factors_new.csv'), parse_dates=True, index_col=['date', 'asset'])
min_prices = pd.read_csv(os.path.join(prices_path, 'min_prices.csv'), index_col=['date'], parse_dates=['date'], date_parser=parser)
min_factors = pd.read_csv(os.path.join(factors_path, 'min_factors_new.csv'), index_col=['date', 'asset'], parse_dates=['date'], date_parser=parser)

因子值分组规则：因子值小的排在第一组，因子值大的排在最后一组
https://github.com/quantopian/alphalens/blob/master/alphalens/examples/alphalens_tutorial_on_quantopian.ipynb
quantiles option chooses the buckets to have the same number of items but it doesn't take into
consideration the factor values. For this reason there is another option bins, which chooses the
buckets to be evenly spaced according to the values themselves.

In [3]:
# min_prices.plot()
# plt.show()
print(f"{daily_prices.index}, \n {min_prices.index}")

DatetimeIndex(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05',
               '2018-01-08', '2018-01-09', '2018-01-10', '2018-01-11',
               '2018-01-12', '2018-01-15',
               ...
               '2021-05-19', '2021-05-20', '2021-05-21', '2021-05-24',
               '2021-05-25', '2021-05-26', '2021-05-27', '2021-05-28',
               '2021-05-31', '2021-06-01'],
              dtype='datetime64[ns]', name='date', length=828, freq=None), 
 DatetimeIndex(['2022-01-17 14:55:00', '2022-01-17 14:56:00',
               '2022-01-17 14:57:00', '2022-01-17 14:58:00',
               '2022-01-17 14:59:00', '2022-01-18 09:00:00',
               '2022-01-18 09:01:00', '2022-01-18 09:02:00',
               '2022-01-18 09:03:00', '2022-01-18 09:04:00',
               ...
               '2022-02-14 14:50:00', '2022-02-14 14:51:00',
               '2022-02-14 14:52:00', '2022-02-14 14:53:00',
               '2022-02-14 14:54:00', '2022-02-14 14:55:00',
               '2022-02-14

In [9]:
print(f"{daily_factors.index.levels[0]}, \n {min_factors.index.levels[0]}")
# A way to solve ValueError: Inferred frequency None from passed values does not conform to passed frequency C #371
# https://github.com/quantopian/alphalens/issues/371
# set freq to C
# min_factors.index.levels[0].freq = 'C'
min_factors.index.levels[0].freq = 'T'
print(f"{daily_factors.index.levels[0]}, \n {min_factors.index.levels[0]}")

DatetimeIndex(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05',
               '2018-01-08', '2018-01-09', '2018-01-10', '2018-01-11',
               '2018-01-12', '2018-01-15',
               ...
               '2021-05-19', '2021-05-20', '2021-05-21', '2021-05-24',
               '2021-05-25', '2021-05-26', '2021-05-27', '2021-05-28',
               '2021-05-31', '2021-06-01'],
              dtype='datetime64[ns]', name='date', length=828, freq=None), 
 DatetimeIndex(['2022-01-17 14:55:00', '2022-01-17 14:56:00',
               '2022-01-17 14:57:00', '2022-01-17 14:58:00',
               '2022-01-17 14:59:00', '2022-01-18 09:00:00',
               '2022-01-18 09:01:00', '2022-01-18 09:02:00',
               '2022-01-18 09:03:00', '2022-01-18 09:04:00',
               ...
               '2022-02-14 14:50:00', '2022-02-14 14:51:00',
               '2022-02-14 14:52:00', '2022-02-14 14:53:00',
               '2022-02-14 14:54:00', '2022-02-14 14:55:00',
               '2022-02-14

ValueError: Inferred frequency None from passed values does not conform to passed frequency T

### compute_forward_returns参数及返回

#### pandas大于1版本，compute_forward_returns报错解决：
1. RuntimeError: Cannot set name on a level of a MultiIndex. Use 'MultiIndex.set_names' instead.
2. 解决办法：https://github.com/quantopian/alphalens/pull/364/files

#### 参数：prices
1. Pricing data must span the factor analysis time period plus an additional buffer window that is greater than the maximum number of expected periods

#### 返回：Returns
1. A MultiIndex DataFrame indexed by timestamp (level 0) and asset (level 1), containing the forward returns for assets.
2. Forward returns column names follow the format accepted by pd.Timedelta (e.g. '1D', '30m', '3h15m', '1D1h', etc).
3. 'date' index freq property (forward_returns.index.levels[0].freq) will be set to a trading calendar (pandas DateOffset) inferred from the input data (see infer_trading_calendar for more details).

In [7]:
# forward_returns = al.utils.compute_forward_returns(factor=daily_factors, 
#                                                    prices=daily_prices, 
#                                                    periods=(1, 3, 5), 
#                                                    filter_zscore=20,
#                                                    cumulative_returns=True)
 
forward_returns = al.utils.compute_forward_returns(factor=min_factors, 
                                                   prices=min_prices, 
                                                   periods=(1, 3, 5), 
                                                   filter_zscore=20,
                                                   cumulative_returns=True)

forward_returns

ValueError: Inferred frequency None from passed values does not conform to passed frequency C

### get_clean_factor_and_forward_returns参数说明

#### groupby，行业分组
1. Either A MultiIndex Series indexed by date and asset,containing the period wise group codes for each asset
2. or a dict of asset to group mappings. If a dict is passed, it is assumed that group mappings are unchanged for the entire time period of the passed factor data.

#### binning_by_group, 是否按照行业分组， bool

#### quantiles, 分组个数
1. Number of equal-sized quantile buckets to use in factor bucketing.
2. Alternately sequence of quantiles, allowing non-equal-sized buckets, e.g. [0, .10, .5, .90, 1.] or [.05, .5, .95]
3. Only one of 'quantiles' or 'bins' can be not-None

#### bins，直方图个数
1. Number of equal-width (valuewise) bins to use in factor bucketing.
2. Alternately sequence of bin edges allowing for non-uniform bin width, e.g. [-4, -2, -0.5, 0, 10]
3. Chooses the buckets to be evenly spaced according to the values themselves. Useful when the factor contains discrete values.

#### periods
1. periods to compute forward returns on.

#### filter_zscore 
1. Sets forward returns greater than X standard deviations from the the mean to nan. Set it to 'None' to avoid filtering.
2. Caution: this outlier filtering incorporates lookahead bias.


### 日频，get_clean_factor_and_forward_returns

In [ ]:
factor_data = al.utils.get_clean_factor_and_forward_returns(factor=factors,
                                                            prices=prices,
                                                            groupby=None,
                                                            binning_by_group=False,
                                                            quantiles=5,  # None
                                                            bins=None,  # [0, 0.2, 0.4, 0.8, 1.0]
                                                            periods=(1, 3, 5),
                                                            filter_zscore=20,
                                                            groupby_labels=None,
                                                            max_loss=0.35,
                                                            zero_aware=False,
                                                            cumulative_returns=True)

In [ ]:
al.tears.create_full_tear_sheet(factor_data, long_short=True, group_neutral=False, by_group=False)

In [ ]:
al.tears.create_event_returns_tear_sheet(factor_data, prices,
                                         avgretplot=(5, 15),
                                         long_short=True,
                                         group_neutral=False,
                                         std_bar=True,
                                         by_group=False)

In [ ]:
al.tears.create_summary_tear_sheet(factor_data, long_short=True, group_neutral=False)

In [ ]:
al.tears.create_returns_tear_sheet(factor_data, long_short=True, group_neutral=False, by_group=False)

In [ ]:
al.tears.create_information_tear_sheet(factor_data, group_neutral=False, by_group=False)

In [ ]:
al.tears.create_turnover_tear_sheet(factor_data)

In [ ]:
ic = al.performance.mean_information_coefficient(factor_data, by_time='1y')

attr = ic.index.strftime('%Y')
v1 = list(ic['1D'].round(2))
v2 = list(ic['3D'].round(2))
v3 = list(ic['5D'].round(2))
bar = Bar('IC均值：2006-2019')
bar.add('1D', attr, v1)
bar.add('3D', attr, v2)
bar.add('5D', attr, v3)
bar.render()


In [ ]:
factor_data